In [1]:
import pandas as pd
import numpy as np

In [2]:
df_sigfe = pd.read_excel('input\\Fonasa cxc LM..xlsx')
df_licencias = pd.read_excel('input\\licencias_medicas_resumen.xlsx', header = 1)

c:\Users\gmolina\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:211: UserWarning: Cell F11437 is marked as a date but the serial value 3112018 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
c:\Users\gmolina\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:211: UserWarning: Cell F11438 is marked as a date but the serial value 20367999 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
c:\Users\gmolina\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:211: UserWarning: Cell F11439 is marked as a date but the serial value 23772353 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
c:\Users\gmolina\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:211: UserWarning: Cell F11440 is marked as a date but the serial value 23774195 is outside t

In [3]:
df_licencias_correctas = df_licencias[df_licencias['F. Inicio'].notna()]

df_licencias_erroneas = df_licencias[df_licencias['F. Inicio'].isna()].drop(columns = ['N°'])
df_licencias_erroneas = df_licencias_erroneas.dropna(axis = 0, how = 'all')
df_licencias_erroneas['F. Inicio'] = df_licencias_erroneas['Nº Licencia']

df_licencias_total = pd.concat([df_licencias_correctas, df_licencias_erroneas])

## Ahora, hay que hacer el inner join/left join con sigfe

In [4]:
conteo_cantidad_licencias = df_sigfe['Nro. Documento'].value_counts()
df_sigfe.query('`Nro. Documento` == 46214504')
df_sigfe.query('`Nro. Documento` == 4386160')
df_sigfe.query('`Nro. Documento` == 58216624')

,Principal,Principal2,Nro. Documento,Fecha,Año,Tipo de Docto,Monto Disponible
1409,61603000-0,FONDO NACIONAL DE SALUD,58216624,2021-10-31,2021,Licencia Medica,673951
1443,61603000-0,FONDO NACIONAL DE SALUD,58216624,2021-10-31,2021,Licencia Medica,673951


In [5]:
df_licencias_total = df_licencias_total.set_index('Nº Licencia')
df_sigfe = df_sigfe.set_index('Nro. Documento')

In [6]:
unidas = pd.merge(df_sigfe, df_licencias_total, how = 'left', left_index = True, right_index = True, suffixes = ('_sigfe', '_licencias_totales')).reset_index()
unidas = unidas.rename(columns = {'index': 'N° Licencia', 'Fecha': 'Fecha_sigfe'})

In [7]:
columnas = ['N° Licencia', 'Fecha_sigfe', 'Monto Disponible ', 'Rut', 'Nombre', 'F. Inicio', 'F. Termino', 'Valor Calculado']
df_unida_final = unidas[columnas].sort_values('Fecha_sigfe', ascending = False)

with pd.ExcelWriter('licencias_sigfe.xlsx', datetime_format = 'DD-MM-YYYY') as writer:
    df_unida_final.to_excel(writer, index = False)

In [8]:
df_licencias_total = df_licencias_total.reset_index()
df_sigfe = df_sigfe.reset_index()

## Debemos hacer un OUTER JOIN, para ver las licencais que sólo están en SIGFE y que NO están en la planilla de licencias.

- Para esto, se unirán ambas tablas según la llave Nro. Documento + Saldo. 
- En la tabla del sigfe, el saldo está en la columna: "Monto Disponible ". Este es del tipo int64
- En la tabla de las licencias, el saldo está en la columna: "Valor Calculado". Este es del tipo object

In [9]:
df_sigfe = df_sigfe.rename(columns = {'Monto Disponible ': 'Valor Calculado', 'Nro. Documento': 'Nº Licencia'})

In [10]:
df_licencias_total['Valor Calculado'] = df_licencias_total['Valor Calculado'].astype(str) \
                                                                            .str.strip() \
                                                                            .str.replace('nan', '', regex = False) \
                                                                            .str.replace('$', '', regex = False) \
                                                                            .str.replace('-', '', regex = False) \
                                                                            .str.replace('.', '', regex = False) \
                                                                            .replace('', np.nan, regex = False) \
                                                                            .astype('Int64')

In [12]:
df_sigfe['Valor Calculado'] = df_sigfe['Valor Calculado'].astype(str) \
                                                            .str.strip() \
                                                            .str.replace('nan', '', regex = False) \
                                                            .str.replace('$', '', regex = False) \
                                                            .str.replace('-', '', regex = False) \
                                                            .str.replace('.', '', regex = False) \
                                                            .replace('', np.nan, regex = False) \
                                                            .astype('Int64')

In [13]:
llave = ['Nº Licencia', 'Valor Calculado']
df_licencias_llave = df_licencias_total.set_index(keys = llave)
df_sigfe_llave = df_sigfe.set_index(keys = llave)

In [40]:
outer_join = pd.merge(df_sigfe_llave, df_licencias_llave, how = 'left', left_index = True, right_index = True).reset_index()
outer_join = outer_join.rename(columns = {'Fecha': 'Fecha_sigfe'})

In [44]:
columnas = ['Nº Licencia', 'Fecha_sigfe', 'Valor Calculado', 'Rut', 'Nombre', 'F. Inicio', 'F. Termino']
outer_join_matched = outer_join.dropna(subset = ['Rut'])
outer_join_no_matched = outer_join[outer_join['Rut'].isna()]

df_unida_final = outer_join[columnas]
df_unida_final_con_matches = outer_join_matched[columnas]
df_unida_final_sin_matches = outer_join_no_matched[columnas]

In [43]:
with pd.ExcelWriter('licencias_sigfe.xlsx', datetime_format = 'DD-MM-YYYY') as writer:
    df_unida_final.to_excel(writer, index = False, sheet_name = 'licencias_sigfe_totales')
    df_unida_final_con_matches.to_excel(writer, index = False, sheet_name = 'licencias_sigfe_match')
    df_unida_final_sin_matches.to_excel(writer, index = False, sheet_name = 'licencias_sigfe_sin_match')
